In [ ]:
!pip install piper-phonemize

ERROR: Could not find a version that satisfies the requirement piper-phonemize (from versions: none)
ERROR: No matching distribution found for piper-phonemize


In [ ]:
!git clone https://huggingface.co/roshan-research/spacy_pos_tagger_parsbertpostagger

Cloning into 'spacy_pos_tagger_parsbertpostagger'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (21/21), 719.37 KiB | 1.15 MiB/s, done.


In [ ]:
!wget -O /content/pos_tagger.py https://raw.githubusercontent.com/roshan-research/hazm/refs/heads/master/hazm/pos_tagger.py
!echo -e "import spacy\n\nfrom spacy.tokens import Doc\nfrom spacy.tokens import DocBin\nfrom spacy.vocab import Vocab\n\nfrom sklearn.metrics import classification_report,f1_score,accuracy_score,precision_score,recall_score\n\nfrom tqdm import tqdm\n" > /tmp/new_file.py && cat /content/pos_tagger.py >> /tmp/new_file.py && mv /tmp/new_file.py /content/pos_tagger.py
!sed -i "s/self\.tagger(\[' '\.join(\[tok for tok in tokens\])\])/self.tagger(' '.join(tokens))/g" /content/pos_tagger.py

--2025-08-20 07:23:39--  https://raw.githubusercontent.com/roshan-research/hazm/refs/heads/master/hazm/pos_tagger.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28649 (28K) [text/plain]
Saving to: ‘/content/pos_tagger.py’

/content/pos_tagger 100%[===================>]  27.98K  --.-KB/s    in 0.002s  

2025-08-20 07:23:40 (13.0 MB/s) - ‘/content/pos_tagger.py’ saved [28649/28649]



In [ ]:
!git clone https://github.com/roshan-research/hazm.git

# Replace Python version constraint
!sed -i 's/python= ">=3.8, <3.12"/python= ">=3.8"/g' hazm/pyproject.toml

# Replace numpy version constraint
!sed -i 's/numpy = "~1.24"/numpy = "1.26"/g' hazm/pyproject.toml

!pip install ./hazm

Cloning into 'hazm'...
remote: Enumerating objects: 8674, done.
remote: Counting objects: 100% (1801/1801), done.
remote: Compressing objects: 100% (303/303), done.
remote: Total 8674 (delta 1651), reused 1498 (delta 1498), pack-reused 6873 (from 3)
Receiving objects: 100% (8674/8674), 24.46 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (5548/5548), done.
Processing ./hazm
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fasttext_wheel-0.9.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached flashtext-2.7.tar.gz (14 kB)
  Preparing metadata (setup.py) ... done
  Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached python_crfsuite-0.9.11-cp312-cp312-manylinux_2_17_x86_64.manylinux

In [ ]:
!pip install -U  spacy[cuda12x] spacy-transformers numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
  Using cached transformers-4.53.3-py3-none-any.whl.metadata (40 kB)


In [ ]:
import os
import json
import re
import requests
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple, Optional
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForTokenClassification,
    TrainingArguments, Trainer, DataCollatorForTokenClassification
)
import openai
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import re
import random
from typing import List

class PersianTextPreprocessor:
    def __init__(self):
        self.arabic_to_persian = {
            'ك': 'ک',
            'ي': 'ی',
            'ء': '',
            'أ': 'ا',
            'إ': 'ا',
            'آ': 'ا',
            'ة': 'ه',
            'ؤ': 'و',
            'ئ': 'ی',
        }

        self.persian_digits = '۰۱۲۳۴۵۶۷۸۹'
        self.arabic_digits = '٠١٢٣٤٥٦٧٨٩'
        self.english_digits = '0123456789'

        self.diacritics = 'ًٌٍَُِّْٰٕٖٜٟٔٗ٘ٙٚٛٝٞٱٲٳٴٵٶٷٹٺٻټٽٿ'

    def normalize_text(self, text: str) -> str:
        if not text:
            return ""

        for arabic, persian in self.arabic_to_persian.items():
            text = text.replace(arabic, persian)

        for diacritic in self.diacritics:
            text = text.replace(diacritic, '')

        for persian_digit, english_digit in zip(self.persian_digits, self.english_digits):
            text = text.replace(persian_digit, english_digit)

        for arabic_digit, english_digit in zip(self.arabic_digits, self.english_digits):
            text = text.replace(arabic_digit, english_digit)

        text = text.replace('\u200c', '')
        text = text.replace('\u200d', '')
        text = text.replace('\u200e', '')
        text = text.replace('\u200f', '')

        text = re.sub(r'\s+', ' ', text)
        text = text.strip()

        text = re.sub(r'[۔\.]{2,}', '.', text)
        text = re.sub(r'[؟\?]{2,}', '؟', text)
        text = re.sub(r'[!]{2,}', '!', text)

        return text

    def tokenize_words(self, text: str) -> List[str]:
        if not text:
            return []

        text = self.normalize_text(text)

        persian_punctuation = '۔؟!،؍؎؏ؘؙؚؐؑؒؓؔؕؖؗ؛؜؝؞؟'
        english_punctuation = '.,;:!?"\'()[]{}«»""''…-–—'
        all_punctuation = persian_punctuation + english_punctuation

        pattern = f'([{re.escape(all_punctuation)}\\s]+)'

        tokens = re.split(pattern, text)

        words = []
        for token in tokens:
            token = token.strip()
            if token and not all(c in all_punctuation + ' \t\n\r' for c in token):
                if re.search(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\w]', token):
                    words.append(token)

        return words

## open ai dataset

In [ ]:
# class PersianDatasetLoader:
#     def __init__(self):
#         self.preprocessor = PersianTextPreprocessor()

#     def load_persian_datasets(self, max_sentences: int = 200) -> List[str]:
#         sentences = self._get_sample_persian_sentences()
#         sentences = sentences * (max_sentences // len(sentences) + 1)
#         random.shuffle(sentences)

#         normalized_sentences = []
#         for sent in sentences:
#             normalized = self.preprocessor.normalize_text(sent)
#             if 4 <= len(normalized) <= 150:
#                 normalized_sentences.append(normalized)

#         if len(normalized_sentences) > max_sentences:
#             normalized_sentences = random.sample(normalized_sentences, max_sentences)

#         return normalized_sentences[:max_sentences]

#     def _get_sample_persian_sentences(self) -> List[str]:
#         return [
#             # Basic greetings and conversations
#             "سلام، چطور هستید؟",
#             "من خوب هستم، ممنون.",
#             "شما چه کار می‌کنید؟",
#             "من در یک شرکت کار می‌کنم.",
#             "خوشحالم که شما را ملاقات کردم.",
#             "روز شما چطور گذشت؟",
#             "حال شما چطور است؟",
#             "کجا زندگی می‌کنید؟",
#             "با شما آشنا شدم خوشحال شدم.",
#             "امیدوارم روز خوبی داشته باشید.",

#             # Daily activities
#             "امروز هوا بسیار زیبا است.",
#             "من به دانشگاه می‌روم.",
#             "فردا به سینما خواهیم رفت.",
#             "دوستان من در پارک بازی می‌کنند.",
#             "مادرم شیرینی پخت.",
#             "صبحانه مهم‌ترین وعده غذایی روز است.",
#             "بچه‌ها در حیاط مدرسه بازی می‌کنند.",
#             "کودکان با شادی بازی می‌کنند.",
#             "باران پاییزی هوای تازه‌ای می‌آورد.",
#             "من هر روز صبح ورزش می‌کنم.",
#             "شام را با خانواده می‌خوریم.",
#             "کتاب خواندن عادت روزانه من است.",
#             "قدم زدن در پارک آرام‌بخش است.",
#             "تلویزیون تماشا کردن تفریح من است.",
#             "موسیقی گوش دادن آرامم می‌کند.",

#             # Education and learning
#             "استاد درس زیبایی تدریس کرد.",
#             "کتابخانه مکان آرامی برای مطالعه است.",
#             "دانشجویان با علاقه درس می‌خوانند.",
#             "کودکان در مدرسه چیزهای جدید یاد می‌گیرند.",
#             "استادان دانشگاه تجربه زیادی دارند.",
#             "کتاب‌های تاریخی داستان‌های جالبی دارند.",
#             "محققان ایرانی در علوم مختلف فعال هستند.",
#             "درس ریاضی برای من سخت است.",
#             "زبان انگلیسی یادگیری مفید است.",
#             "امتحان فردا خیلی مهم است.",
#             "تکالیف مدرسه را باید انجام دهم.",
#             "مطالعه در کتابخانه بهتر است.",
#             "معلم ما بسیار صبور است.",
#             "دانش آموزان باید سخت درس بخوانند.",
#             "آزمون ورودی دانشگاه سخت است.",

#             # Culture and literature
#             "کتاب‌های فارسی بسیار جالب هستند.",
#             "شعرهای حافظ بسیار زیبا هستند.",
#             "زبان فارسی زبان شعر و ادب است.",
#             "موسیقی سنتی ایران تاریخ غنی دارد.",
#             "هنرمندان ایرانی در سراسر جهان مشهور هستند.",
#             "فرهنگ غنی ایران در تمام جهان شناخته شده است.",
#             "شعرهای فردوسی حماسی هستند.",
#             "سعدی و حافظ شاعران بزرگ ایران هستند.",
#             "شاهنامه شاهکار ادبیات فارسی است.",
#             "موسیقی کلاسیک ایران عمیق است.",
#             "نقاشی مینیاتور هنر ایرانی است.",
#             "خوشنویسی فارسی بسیار زیبا است.",
#             "تعزیه نمایش سنتی ایرانی است.",
#             "فرش ایرانی شاهکار هنر است.",
#             "معماری اسلامی در ایران شکوفا شده است.",

#             # Geography and places
#             "تهران پایتخت ایران است.",
#             "رودخانه کارون طولانی‌ترین رود ایران است.",
#             "کوه‌های البرز منظره‌ای فوق‌العاده دارند.",
#             "باغ‌های شیراز بسیار معروف هستند.",
#             "دریای خزر در شمال ایران قرار دارد.",
#             "شهرهای تاریخی ایران جاذبه‌های توریستی هستند.",
#             "اصفهان نصف جهان نامیده می‌شود.",
#             "شیراز شهر شعر و ادب ایران است.",
#             "مشهد شهر مقدس ایران است.",
#             "تبریز شهر فرش‌های زیبا است.",
#             "کاشان شهر گلاب است.",
#             "یزد شهر بادگیرها است.",
#             "کرمان شهر پسته است.",
#             "گیلان استان برنج است.",
#             "فارس استان تاریخی ایران است.",

#             # Food and hospitality
#             "غذای ایرانی خیلی خوشمزه است.",
#             "چلو کباب غذای محبوب ایرانی است.",
#             "چای ایرانی طعم خاصی دارد.",
#             "خانواده‌های ایرانی بسیار مهمان‌نواز هستند.",
#             "مردم ایران بسیار مهربان و صمیمی هستند.",
#             "قورمه سبزی غذای خوشمزه‌ای است.",
#             "فسنجان غذای شمالی ایران است.",
#             "آش رشته غذای محبوب زمستانی است.",
#             "کوکو سبزی غذای بهاری است.",
#             "باقلا پلو با محوه غذای گیلانی است.",
#             "خورشت بامیه غذای خانگی است.",
#             "دوغ نوشیدنی سنتی ایران است.",
#             "شیرینی یزدی بسیار خوشمزه است.",
#             "بستنی سنتی شیراز معروف است.",
#             "حلوا شیرینی محبوب ایرانی است.",
#         ]

In [ ]:
# OPENAI_API_KEY = ""

In [ ]:
# import openai
# import json
# from typing import Dict, List
# from pydantic import BaseModel

# class EzafeLabels(BaseModel):
#     needs_ezafe: List[bool]

# class OpenAIEzafeLabeler:
#     def __init__(self, api_key: str):
#         self.client = openai.OpenAI(api_key=api_key)
#         self.preprocessor = PersianTextPreprocessor()

#     def label_sentence(self, sentence: str) -> Dict:
#         words = self.preprocessor.tokenize_words(sentence)

#         system_prompt = f"""
# تو یک متخصص زبان فارسی هستی. فهرست کلمات زیر را تحلیل کن و مشخص کن کدام کلمات نیاز به "اِ ربط" دارند.

# قوانین:
# - اِ ربط (کسره): بین صفت و موصوف، مضاف و مضاف‌الیه استفاده می‌شود
# - فقط کلماتی که به کلمه بعدی متصل می‌شوند نیاز به اِ ربط دارند
# - آخرین کلمه هیچ‌وقت نیاز به اِ ربط ندارد
# - باید دقیقاً به تعداد کلمات ورودی، جواب true/false بدهی

# مثال‌ها:

# مثال 1:
# کلمات: ["کتاب", "قرمز", "دانشجو"]
# needs_ezafe: [true, true, false]
# توضیح: "کتاب" نیاز به اِ ربط دارد (کتابِ قرمز)، "قرمز" نیاز به اِ ربط دارد (قرمزِ دانشجو)، "دانشجو" نیاز ندارد.

# مثال 2:
# کلمات: ["خانه", "بزرگ", "مادربزرگ", "من"]
# needs_ezafe: [true, true, true, false]
# توضیح: "خانه" نیاز دارد (خانه‌ی بزرگ)، "بزرگ" نیاز دارد (بزرگِ مادربزرگ)، "مادربزرگ" نیاز دارد (مادربزرگِ من)، "من" نیاز ندارد.

# مثال 3:
# کلمات: ["دختر", "زیبا", "خواب", "است"]
# needs_ezafe: [true, false, false, false]
# توضیح: "دختر" نیاز دارد (دخترِ زیبا)، بقیه نیاز ندارند.

# مثال 4:
# کلمات: ["درس", "ریاضی", "سخت", "است"]
# needs_ezafe: [true, false, false, false]
# توضیح: "درس" نیاز دارد (درسِ ریاضی)، بقیه نیاز ندارند.

# مثال 5:
# کلمات: ["کتابخانه", "دانشگاه", "تهران", "بزرگ", "است"]
# needs_ezafe: [true, true, false, false, false]
# توضیح: "کتابخانه" نیاز دارد (کتابخانه‌ی دانشگاه)، "دانشگاه" نیاز دارد (دانشگاهِ تهران)، بقیه نیاز ندارند.

# مثال 6:
# کلمات: ["شعرهای", "حافظ", "زیبا", "هستند"]
# needs_ezafe: [true, false, false, false]
# توضیح: "شعرهای" نیاز دارد (شعرهایِ حافظ)، بقیه نیاز ندارند.

# مثال 7:
# کلمات: ["استاد", "درس", "زیبایی", "تدریس", "کرد"]
# needs_ezafe: [false, true, false, false, false]
# توضیح: "درس" نیاز دارد (درسِ زیبایی)، بقیه نیاز ندارند.

# مثال 8:
# کلمات: ["موسیقی", "سنتی", "ایران", "زیبا", "است"]
# needs_ezafe: [true, true, false, false, false]
# توضیح: "موسیقی" نیاز دارد (موسیقیِ سنتی)، "سنتی" نیاز دارد (سنتیِ ایران)، بقیه نیاز ندارند.

# حالا تعداد کلمات ورودی: {len(words)}
# باید دقیقاً {len(words)} مقدار true/false برگردانی.
# """

#         words_str = str(words)

#         response = self.client.beta.chat.completions.parse(
#             model="gpt-4.1",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": f"جمله: {sentence}\nکلمات: {words_str}"}
#             ],
#             response_format=EzafeLabels,
#             temperature=0.1,
#             max_tokens=500
#         )

#         labels = response.choices[0].message.parsed.needs_ezafe

#         if len(labels) != len(words):
#             raise Exception(f"Mismatch in length! Words: {len(words)}, Labels: {len(labels)}")

#         return {
#             "sentence": sentence,
#             "words": words,
#             "labels": labels,
#         }

In [ ]:
# dataset_loader = PersianDatasetLoader()
# sentences = dataset_loader.load_persian_datasets(max_sentences=100)

In [ ]:
# from tqdm import tqdm

# labeler = OpenAIEzafeLabeler(OPENAI_API_KEY)
# labeled_data = []

# for sentence in tqdm(sentences):
#     try:
#       labeled_item = labeler.label_sentence(sentence)
#       labeled_data.append(labeled_item)
#     except:
#       pass

## HomoRich Dataset

In [ ]:
import pandas as pd
import itertools
import random
from typing import List, Dict

FILE_URLS = [
    "https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_01.parquet",
    "https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_02.parquet",
    "https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_03.parquet"
]

def load_homorich(max_sentences: int = 20_000,
                  file_urls=FILE_URLS,
                  shuffle=True,
                  seed=42) -> pd.DataFrame:
    def read_shard(url: str) -> pd.DataFrame:
        print(f"Loading {url}")
        df = pd.read_parquet(url)

        return df[["Grapheme", "Phoneme"]]

    frames = [read_shard(url) for url in file_urls]
    df = pd.concat(frames, ignore_index=True)

    if shuffle:
        df = df.sample(frac=1, random_state=seed)

    return df.head(max_sentences)

In [ ]:
homorich_df = load_homorich(max_sentences=50_000)
homorich_df

Loading https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_01.parquet
Loading https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_02.parquet
Loading https://huggingface.co/datasets/MahtaFetrat/HomoRich-G2P-Persian/resolve/main/data/part_03.parquet


,Grapheme,Phoneme
446698,سی و یک، سی و دو، سی و سه.,si ?o yek si ?o do si ?o se
310616,این قطعه از داستان را خیلی دوست دارم.,?in qat?e ?az dAstAn rA xeyli dust dAram
428128,آیا می توانم چک شخصی بدهم؟,?AyA mitavAnam Ceke Saxsi bedaham?
139706,یخچال پر از خوراکی‌های تازه و سالم بود.,yaxCAl por ?az xorAkihAye tAze va sAlem bud
438997,شستم خبر دار شد که کلکی در کار است.,Sostam xabar dAr Sod ke kalaki dar kAr ?ast
...,...,...
328674,دوست داریم غذاهای اونجا رو امتحان کنیم .,dust dArim qAzAhAye ?unjA ro ?emtehAn konim
103746,او در میان یادداشت‌های قدیمی‌اش ماند و به خاطر...,?u dar miyAne yAddASthAye qadimi?aS mAnd va be...
434030,آیا هیچ بیماری های مزمن دارید؟,?AyA hiC bimArihAye mozmen dArid?
232107,او با اجرای خود، جشنواره را می‌ترکاند.,?u bA ?ejrAye xod jaSnvAre rA miterekAnd


In [ ]:
from pos_tagger import SpacyPOSTagger

spacy_posTagger = SpacyPOSTagger(model_path='spacy_pos_tagger_parsbertpostagger', using_gpu=True)
preprocessor = PersianTextPreprocessor()

------------------ GPU Setup Process Started ---------------------
------------ GPU is available and ready for use -------------


In [ ]:
from hazm import word_tokenize
from tqdm import tqdm

def infer_ezafe_labels(words: List[str], phonemes: List[str]) -> List[bool]:
    labels = []
    for i, ph in enumerate(phonemes):
        if words[i].endswith("ه") or i == len(phonemes) - 1:
            labels.append(False)
        else:
            labels.append(ph.endswith("e") or ph.endswith("ye"))
    return labels

def infer_spacy_labels(sent: str) -> Tuple[List[str], List[bool]]:
    words = word_tokenize(sent)
    tagged = spacy_posTagger.tag(tokens=words, universal_tag=False)

    spacy_words = []
    ezafe_labels = []

    for word, pos_tag in tagged:
        spacy_words.append(word)
        ezafe_labels.append(',EZ' in pos_tag)

    return spacy_words, ezafe_labels



In [ ]:
labeled_data = homorich_to_labeled_list(homorich_df)
print(f"Kept {len(labeled_data):,} clean-aligned sentences")

Processing sentences: 100%|██████████| 50000/50000 [10:01<00:00, 83.18it/s]

Kept 49,656 clean-aligned sentences


## verify and save dataset

In [ ]:
for item in labeled_data[5:15]:
    print(f"جمله: {item['sentence']}")
    print(f"کلمات: {item['words']}")
    print(f"برچسب‌ها: {item['labels']}")
    print(f"تعداد کلمات: {len(item['words'])}")
    print(f"تعداد برچسب‌ها: {len(item['labels'])}")

    print("تحلیل کلمه به کلمه:")
    for word, needs_ezafe in zip(item['words'], item['labels']):
        ezafe_indicator = "✓" if needs_ezafe else "✗"
        print(f"  {word}: {ezafe_indicator}")
    print("-" * 60)

جمله: در این پروژه طراحی، ایده‌های نوآورانه‌ای گنجاندند که تحسین‌برانگیز بود.
کلمات: ['در', 'این', 'پروژه', 'طراحی', '،', 'ایده\u200cهای', 'نوآورانه\u200cای', 'گنجاندند', 'که', 'تحسین\u200cبرانگیز', 'بود', '.']
برچسب‌ها: [False, False, True, False, False, True, False, False, False, False, False, False]
تعداد کلمات: 12
تعداد برچسب‌ها: 12
تحلیل کلمه به کلمه:
  در: ✗
  این: ✗
  پروژه: ✓
  طراحی: ✗
  ،: ✗
  ایده‌های: ✓
  نوآورانه‌ای: ✗
  گنجاندند: ✗
  که: ✗
  تحسین‌برانگیز: ✗
  بود: ✗
  .: ✗
------------------------------------------------------------
جمله: علی سونا می رفت و من بخور می دادم 
کلمات: ['علی', 'سونا', 'می', 'رفت', 'و', 'من', 'بخور', 'می', 'دادم']
برچسب‌ها: [False, False, False, False, False, False, False, False, False]
تعداد کلمات: 9
تعداد برچسب‌ها: 9
تحلیل کلمه به کلمه:
  علی: ✗
  سونا: ✗
  می: ✗
  رفت: ✗
  و: ✗
  من: ✗
  بخور: ✗
  می: ✗
  دادم: ✗
------------------------------------------------------------
جمله: برخی گیاهان بدون نیاز به حشرات گرده‌افشانی می‌کنند.
کلمات: ['بر

In [ ]:
!mkdir drive
!mkdir drive/MyDrive

In [ ]:
import json

with open('/content/drive/MyDrive/labeled_data.json', 'w', encoding='utf-8') as f:
    json.dump(labeled_data, f, ensure_ascii=False, indent=2)

## training

In [ ]:
class EzafeDataset(Dataset):
    def __init__(self, labeled_data, tokenizer, max_length=128):
        self.data = labeled_data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        words = item['words']
        labels = item['labels']

        label_ids = [1 if label else 0 for label in labels]

        tokenized = self.tokenizer(
            words,
            is_split_into_words=True,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        word_ids = tokenized.word_ids()
        aligned_labels = []

        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)  # Ignore padding tokens
            elif word_idx != previous_word_idx:
                if word_idx < len(label_ids):
                    aligned_labels.append(label_ids[word_idx])
                else:
                    aligned_labels.append(-100)
            else:
                aligned_labels.append(-100)  # Ignore subword tokens
            previous_word_idx = word_idx

        return {
            'input_ids': tokenized['input_ids'].squeeze(),
            'attention_mask': tokenized['attention_mask'].squeeze(),
            'labels': torch.tensor(aligned_labels, dtype=torch.long)
        }

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification

model_name = "HooshvareLab/albert-fa-zwnj-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

config = AutoConfig.from_pretrained(model_name)
config.hidden_dropout_prob = 0.2
config.attention_probs_dropout_prob = 0.2

config.num_labels = 2
config.id2label = {0: "NO_EZAFE", 1: "NEEDS_EZAFE"}
config.label2id = {"NO_EZAFE": 0, "NEEDS_EZAFE": 1}

model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    config=config
)

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/albert-fa-zwnj-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(labeled_data, test_size=0.2, random_state=42)

train_dataset = EzafeDataset(train_data, tokenizer)
val_dataset = EzafeDataset(val_data, tokenizer)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/ezafe_model',
    num_train_epochs=4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=400,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    fp16=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to=[],
)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    flat_predictions = [item for sublist in true_predictions for item in sublist]
    flat_labels = [item for sublist in true_labels for item in sublist]

    precision, recall, f1, _ = precision_recall_fscore_support(flat_labels, flat_predictions, average='weighted')
    accuracy = accuracy_score(flat_labels, flat_predictions)

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

model.save_pretrained('/content/drive/MyDrive/ezafe_model_final')
tokenizer.save_pretrained('/content/drive/MyDrive/ezafe_model_final')

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.440900,0.305491,0.846641,0.799880,0.838880,0.846641
400,0.238800,0.121600,0.957271,0.957135,0.957024,0.957271
600,0.172300,0.084419,0.968852,0.968933,0.969031,0.968852
800,0.141400,0.065956,0.976498,0.976456,0.976421,0.976498
1000,0.117100,0.058259,0.979018,0.978981,0.978951,0.979018
1200,0.107500,0.054133,0.980922,0.980971,0.981037,0.980922
1400,0.096400,0.055166,0.981379,0.981437,0.981520,0.981379
1600,0.082900,0.050628,0.982528,0.982585,0.982670,0.982528
1800,0.083900,0.046739,0.984112,0.984100,0.984089,0.984112
2000,0.078200,0.045121,0.984474,0.984470,0.984466,0.984474


('/content/drive/MyDrive/ezafe_model_final/tokenizer_config.json',
 '/content/drive/MyDrive/ezafe_model_final/special_tokens_map.json',
 '/content/drive/MyDrive/ezafe_model_final/spiece.model',
 '/content/drive/MyDrive/ezafe_model_final/added_tokens.json',
 '/content/drive/MyDrive/ezafe_model_final/tokenizer.json')

In [ ]:
!ls /content/drive/MyDrive/ezafe_model_final

config.json	   special_tokens_map.json  tokenizer_config.json
model.safetensors  spiece.model		    tokenizer.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive2')

Mounted at /content/drive2


In [ ]:
!mv /content/drive/MyDrive/ezafe_model_final /content/drive2/MyDrive/ezafe_model_final

In [ ]:
def predict_ezafe(text, model, tokenizer):
    preprocessor = PersianTextPreprocessor()
    words = preprocessor.tokenize_words(text)

    inputs = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    device = model.device
    inputs_on_device = {k: v.to(device) if hasattr(v, 'to') else v for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs_on_device)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_labels = torch.argmax(predictions, dim=-1)

    word_ids = inputs.word_ids()
    word_predictions = []

    previous_word_idx = None
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != previous_word_idx:
            if word_idx < len(words):
                word_predictions.append({
                    'word': words[word_idx],
                    'needs_ezafe': bool(predicted_labels[0][i].item()),
                    'confidence': float(predictions[0][i][predicted_labels[0][i]].item())
                })
            previous_word_idx = word_idx

    return word_predictions

In [ ]:
results

[{'word': 'سنگینی', 'needs_ezafe': True, 'confidence': 0.9616780281066895},
 {'word': 'موفقیت', 'needs_ezafe': False, 'confidence': 0.9805247783660889},
 {'word': 'جای', 'needs_ezafe': True, 'confidence': 0.9994866847991943},
 {'word': 'خود', 'needs_ezafe': False, 'confidence': 0.9997774958610535},
 {'word': 'را', 'needs_ezafe': False, 'confidence': 0.9999128580093384},
 {'word': 'به', 'needs_ezafe': False, 'confidence': 0.9999004602432251},
 {'word': 'سبکی', 'needs_ezafe': False, 'confidence': 0.996906578540802},
 {'word': 'تازهکار', 'needs_ezafe': False, 'confidence': 0.9941118359565735},
 {'word': 'بودن', 'needs_ezafe': False, 'confidence': 0.9993650317192078},
 {'word': 'داده', 'needs_ezafe': False, 'confidence': 0.9999490976333618},
 {'word': 'بود', 'needs_ezafe': False, 'confidence': 0.9997958540916443},
 {'word': 'حالا', 'needs_ezafe': False, 'confidence': 0.9999053478240967},
 {'word': 'نسبتبه', 'needs_ezafe': False, 'confidence': 0.9978384375572205},
 {'word': 'همه', 'needs_ez

In [ ]:
test_sentence = "سنگینی موفقیت جای خود را به سبُکی تازه‌کار بودن داده بود. حالا نسبت‌به همه چیز کمتر مطمئن بودم و همین باعث شد وارد یکی از خلاقانه‌ترین دوره‌های زندگی‌ام شوم."
results = predict_ezafe(test_sentence, model, tokenizer)
print(f"\nTest sentence: {test_sentence}")
for result in results:
    ezafe_indicator = "✓" if result['needs_ezafe'] else "✗"
    print(f"{result['word']}: {ezafe_indicator} (confidence: {result['confidence']:.3f})")


Test sentence: سنگینی موفقیت جای خود را به سبُکی تازه‌کار بودن داده بود. حالا نسبت‌به همه چیز کمتر مطمئن بودم و همین باعث شد وارد یکی از خلاقانه‌ترین دوره‌های زندگی‌ام شوم.
سنگینی: ✓ (confidence: 0.962)
موفقیت: ✗ (confidence: 0.981)
جای: ✓ (confidence: 0.999)
خود: ✗ (confidence: 1.000)
را: ✗ (confidence: 1.000)
به: ✗ (confidence: 1.000)
سبکی: ✗ (confidence: 0.997)
تازهکار: ✗ (confidence: 0.994)
بودن: ✗ (confidence: 0.999)
داده: ✗ (confidence: 1.000)
بود: ✗ (confidence: 1.000)
حالا: ✗ (confidence: 1.000)
نسبتبه: ✗ (confidence: 0.998)
همه: ✗ (confidence: 0.963)
چیز: ✗ (confidence: 0.948)
کمتر: ✗ (confidence: 0.998)
مطمین: ✗ (confidence: 1.000)
بودم: ✗ (confidence: 1.000)
و: ✗ (confidence: 1.000)
همین: ✗ (confidence: 1.000)
باعث: ✗ (confidence: 0.999)
شد: ✗ (confidence: 1.000)
وارد: ✓ (confidence: 0.958)
یکی: ✗ (confidence: 1.000)
از: ✗ (confidence: 1.000)
خلاقانهترین: ✗ (confidence: 0.998)
دورههای: ✓ (confidence: 0.998)
زندگیام: ✗ (confidence: 0.996)
شوم: ✗ (confidence: 1.000)


## export quantized onnx

In [ ]:
!optimum-cli export onnx \
  --model /content/drive/MyDrive/ezafe_model_final \
  --task token-classification \
  --opset 20 \
  /content/drive/MyDrive/ezafe_model_onnx

2025-06-05 15:38:03.759758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749137883.781509   23781 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749137883.788302   23781 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
!optimum-cli onnxruntime quantize \
  --avx512 \
  --onnx_model /content/drive/MyDrive/ezafe_model_onnx \
  -o /content/drive/MyDrive/ezafe_model_quantized

2025-06-05 15:38:23.055933: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749137903.075603   23875 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749137903.081659   23875 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
%cd /content/drive/MyDrive/
!zip -r ezafe_model_final.zip ezafe_model_final/
!zip -r ezafe_model_onnx.zip ezafe_model_onnx/
!zip -r ezafe_model_quantized.zip ezafe_model_quantized/
%cd /content/

/content/drive/MyDrive
  adding: ezafe_model_final/ (stored 0%)
  adding: ezafe_model_final/config.json (deflated 52%)
  adding: ezafe_model_final/model.safetensors (deflated 7%)
  adding: ezafe_model_final/tokenizer_config.json (deflated 95%)
  adding: ezafe_model_final/special_tokens_map.json (deflated 85%)
  adding: ezafe_model_final/spiece.model (deflated 54%)
  adding: ezafe_model_final/tokenizer.json (deflated 75%)
  adding: ezafe_model_onnx/ (stored 0%)
  adding: ezafe_model_onnx/config.json (deflated 52%)
  adding: ezafe_model_onnx/tokenizer_config.json (deflated 95%)
  adding: ezafe_model_onnx/special_tokens_map.json (deflated 85%)
  adding: ezafe_model_onnx/spiece.model (deflated 54%)
  adding: ezafe_model_onnx/tokenizer.json (deflated 75%)
  adding: ezafe_model_onnx/model.onnx (deflated 8%)
  adding: ezafe_model_quantized/ (stored 0%)
  adding: ezafe_model_quantized/model_quantized.onnx (deflated 12%)
  adding: ezafe_model_quantized/ort_config.json (deflated 56%)
  adding: e

## test quantized version

In [ ]:
final_model_path = "/content/drive/MyDrive/ezafe_model_final"
quantized_model_path = "/content/drive/MyDrive/ezafe_model_quantized"

In [ ]:
from optimum.onnxruntime import ORTModelForTokenClassification

quantized_model = ORTModelForTokenClassification.from_pretrained(quantized_model_path)
tokenizer = AutoTokenizer.from_pretrained(final_model_path)

In [ ]:
def predict_ezafe_simple(text, model, tokenizer):
    words = text.split()

    inputs = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_labels = torch.argmax(predictions, dim=-1)

    word_ids = inputs.word_ids()
    results = []

    previous_word_idx = None
    for i, word_idx in enumerate(word_ids):
        if word_idx is not None and word_idx != previous_word_idx:
            if word_idx < len(words):
                results.append({
                    'word': words[word_idx],
                    'needs_ezafe': bool(predicted_labels[0][i].item()),
                    'confidence': float(predictions[0][i][predicted_labels[0][i]].item())
                })
            previous_word_idx = word_idx

    return results

In [ ]:
test_sentences = [
    "سنگینی موفقیت جای خود را به سبُکی تازه‌کار بودن داده بود.",
    "حالا نسبت‌به همه چیز کمتر مطمئن بودم و همین باعث شد وارد یکی از خلاقانه‌ترین دوره‌های زندگی‌ام شوم.",
    "این یک جمله آزمایشی برای تست سرعت و دقت مدل است.",
    "کتاب‌های علمی دانشگاه بسیار مفید هستند.",
    "دانشجویان ایرانی در رشته‌های مختلف تحصیل می‌کنند."
]

for test_sentence in test_sentences:
    results = predict_ezafe_simple(test_sentence, quantized_model, tokenizer)

    print(f"Test sentence: {test_sentence}\n")
    print("Ezafe predictions:")
    for result in results:
        ezafe_indicator = "✓" if result['needs_ezafe'] else "✗"
        print(f"{result['word']}: {ezafe_indicator} (confidence: {result['confidence']:.3f})")
    print()
    print()

Test sentence: سنگینی موفقیت جای خود را به سبُکی تازه‌کار بودن داده بود.

Ezafe predictions:
سنگینی: ✓ (confidence: 0.989)
موفقیت: ✗ (confidence: 0.994)
جای: ✓ (confidence: 1.000)
خود: ✗ (confidence: 1.000)
را: ✗ (confidence: 1.000)
به: ✗ (confidence: 1.000)
سبُکی: ✗ (confidence: 0.994)
تازه‌کار: ✗ (confidence: 0.996)
بودن: ✗ (confidence: 0.999)
داده: ✗ (confidence: 1.000)
بود.: ✗ (confidence: 1.000)


Test sentence: حالا نسبت‌به همه چیز کمتر مطمئن بودم و همین باعث شد وارد یکی از خلاقانه‌ترین دوره‌های زندگی‌ام شوم.

Ezafe predictions:
حالا: ✗ (confidence: 1.000)
نسبت‌به: ✗ (confidence: 0.515)
همه: ✗ (confidence: 0.980)
چیز: ✗ (confidence: 0.996)
کمتر: ✗ (confidence: 0.999)
مطمئن: ✗ (confidence: 0.998)
بودم: ✗ (confidence: 1.000)
و: ✗ (confidence: 1.000)
همین: ✗ (confidence: 1.000)
باعث: ✗ (confidence: 1.000)
شد: ✗ (confidence: 1.000)
وارد: ✓ (confidence: 0.998)
یکی: ✗ (confidence: 1.000)
از: ✗ (confidence: 1.000)
خلاقانه‌ترین: ✗ (confidence: 0.999)
دوره‌های: ✓ (confidence: 0.999)
زندگی